# Train the model on the created dataset

After capturing the images and retrieving the ground truth the next step is to train an object detection model. In this notebook, we will use the [`detectron2`](https://github.com/facebookresearch/detectron2) library to train a Faster-RCNN model.

> **Note**: This notebook is meant to be used with Kaggle notebooks (and also Colab), the link is [here](https://www.kaggle.com/code/leonardodanelutti/trainfasterrcnn). From Kaggle the notebook can be opened in Colab.

## Install detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, shutil, distutils.core
# If detectron2 is already downloaded remove it because seems to cause issues 
if os.path.isdir('./detectron2'):
    shutil.rmtree("./detectron2")
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Set `torch` and `CUDA` vertions

In [ ]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

### Import the required libraries

In [3]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg

## Register the dataset

With the `register_coco_instances` function we can register a COCO-formatted dataset, the arguments for the function are in order:
- name (str): the name that identifies a dataset, e.g. "coco_2014_train".
- metadata (dict): extra metadata associated with this dataset.
- json_file (str): path to the json instance annotation file.
- image_root (str or path-like): directory which contains all the images.

In [4]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("person_park_train", {}, "/kaggle/input/personparktrain/dataset_training.json", "/kaggle/input/personparktrain/lit/lit")

## Set parameters for training

The model used is [Faster-RCNN](https://arxiv.org/abs/1506.01497), with weights and parameters initalized by the R101-FPN baseline (ResNet-101 and FPNs). Most of the parameters are left with the defaults values, more about the parameters can be found [here](https://detectron2.readthedocs.io/en/latest/modules/config.html#config-references).

In [5]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
# Set model to Faster-RCNN for object detection based on ResNet-101 and FPNs.
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
# The datasets to train on
cfg.DATASETS.TRAIN = ("person_park_train",)
# The model is evalueted afterwards 
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
# Let training initialize from model we choose
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
# Decrease the number of images per batch because of the limited GPU memory
cfg.SOLVER.IMS_PER_BATCH = 10
# The number of iteraitons
cfg.SOLVER.MAX_ITER = 10000
cfg.SOLVER.STEPS = (7500,)
# The dataset has only one class
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

#print(cfg)

## Start training the model!

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# Save config
with open("./cfg.yaml", 'w') as file:
    file.write(cfg.dump())
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()